In [36]:
#Carga de las librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

In [37]:
pd.set_option('display.max_columns', None)

In [38]:
df = pd.read_csv('hawaii_limpio_completo2.csv')


In [39]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5269,https://www.airbnb.com/rooms/5269,20250306035046,2025-03-14,city scrape,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","We are located on the ""sunny side"" of Waimea, ...",https://a0.muscache.com/pictures/5b52b72f-5a09...,7620,https://www.airbnb.com/users/show/7620,Lea & Pat,2009-02-09,"Waimea, HI","Lea, born and raised on the Island of Hawaii, ...",within an hour,100%,96%,t,https://a0.muscache.com/im/pictures/user/16ac6...,https://a0.muscache.com/im/pictures/user/16ac6...,Island of Hawaiʻi,3.0,6.0,"['email', 'phone']",t,t,Neighborhood highlights,South Kohala,Hawaii,20.02740,-155.70200,Entire rental unit,Entire home/apt,2,1,1,1,"[""Coffee maker"", ""Garden view"", ""Outdoor furni...",128.0,3,1125,3,3,1125,1125,3.0,1125.0,t,8,8,8,8,2025-03-14,40,6,1,0,6,36,4608.000000,2011-05-31,2025-02-25,4.78,4.88,4.55,4.95,4.85,5.00,4.85,119-269-5808-01R,f,3,3,0,0,0.24
1,5387,https://www.airbnb.com/rooms/5387,20250306035046,2025-03-15,city scrape,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,IN a Farm belt area with small commercial farm...,https://a0.muscache.com/pictures/1170713/dca6a...,7878,https://www.airbnb.com/users/show/7878,Edward,2009-02-13,"Captain Cook, HI","Kona Hawaii,\n \n Ed is a Farmer/Designer \n\n...",within an hour,100%,96%,f,https://a0.muscache.com/im/pictures/user/b895b...,https://a0.muscache.com/im/pictures/user/b895b...,Kailua/Kona,2.0,3.0,"['email', 'phone']",t,t,Neighborhood highlights,South Kona,Hawaii,19.43081,-155.88069,Entire rental unit,Entire home/apt,2,1,0,2,"[""Garden view"", ""Cooking basics"", ""Wifi"", ""Ded...",97.0,5,60,5,5,1125,1125,5.0,1125.0,t,2,22,48,250,2025-03-15,231,13,0,207,14,130,12610.000000,2010-02-16,2025-01-14,4.67,4.69,4.42,4.85,4.88,4.74,4.78,TA-163-133-0304-01,t,2,2,0,0,1.26
2,5480,https://www.airbnb.com/rooms/5480,20250306035046,2025-03-14,city scrape,Isle Of You Naturally Farm Retreat,The Best Choice for your Clothing Optional Nud...,We are located on a rural one lane road going ...,https://a0.muscache.com/pictures/75530989/8ed3...,8145,https://www.airbnb.com/users/show/8145,Normand,2009-02-18,"Pāhoa, HI","a decade ago, my husband bailed on the mainla...",Sin respuesta,0%,0%,f,https://a0.muscache.com/im/users/8145/profile_...,https://a0.muscache.com/im/users/8145/profile_...,Island of Hawaiʻi,3.0,3.0,"['email', 'phone']",t,f,Neighborhood highlights,Puna,Hawaii,19.44149,-154.91527,Private room in cabin,Private room,2,1,1,1,"[""Kitchen"", ""Host greets you"", ""Breakfast"", ""F...",117.0,2,60,2,2,60,60,2.0,60.0,t,30,60,90,365,2025-03-14,0,0

# Seleción y transformación de variables

In [40]:
variables = df[['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'property_type', 'room_type', 'license', 'number_of_reviews_ltm', 'calculated_host_listings_count', 'estimated_occupancy_l365d', 'price']].copy()

In [41]:
variables.shape

(31208, 14)

In [42]:
variables.head()

,host_is_superhost,host_has_profile_pic,host_identity_verified,instant_bookable,host_response_time,host_response_rate,host_acceptance_rate,property_type,room_type,license,number_of_reviews_ltm,calculated_host_listings_count,estimated_occupancy_l365d,price
0,t,t,t,f,within an hour,100%,96%,Entire rental unit,Entire home/apt,119-269-5808-01R,6,3,36,128.0
1,f,t,t,t,within an hour,100%,96%,Entire rental unit,Entire home/apt,TA-163-133-0304-01,13,2,130,97.0
2,f,t,f,f,Sin respuesta,0%,0%,Private room in cabin,Private room,Sin licencia,0,3,0,117.0
3,f,t,f,f,Sin respuesta,0%,0%,Entire home/apt,Entire home/apt,Sin licencia,0,1,0,803.5
4,f,t,t,f,within a day,100%,0%,Entire home,Entire home/apt,GE-104-390-7584-01,0,1,0,98.0


In [43]:
#Transformación de host_response_time
variables.host_response_time.unique()

array(['within an hour', 'Sin respuesta', 'within a day',
       'within a few hours', 'a few days or more'], dtype=object)

In [44]:
variables['host_response_time'] = variables['host_response_time'].replace(['within an hour', 'within a few hours'], 'within a day')
variables['host_response_time'] = variables['host_response_time'].replace('Sin respuesta', 'a few days or more')


In [45]:
#Convertir a formato numérico host_response_rate y host_acceptance_rate
variables['host_response_rate'] = variables['host_response_rate'].str.rstrip('%').astype('float') / 100.0
variables['host_acceptance_rate'] = variables['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0


In [46]:
#transformación host response rate
variables['host_response_rate'] = variables['host_response_rate'].apply(lambda x: '90% o más' if x >= 0.90 else 'Menos de 90%')


In [47]:
#Transformación host acceptance rate
variables['host_acceptance_rate'] = variables['host_acceptance_rate'].apply(lambda x: '80% o más' if x >= 0.80 else 'Menos de 80%')

In [48]:
variables['property_type'].value_counts()

property_type
Entire condo                  15198
Entire rental unit             5994
Entire home                    2686
Room in hotel                  1240
Private room in resort         1103
                              ...  
Barn                              1
Tower                             1
Shared room in tent               1
Private room in camper/rv         1
Shared room in rental unit        1
Name: count, Length: 80, dtype: int64

In [49]:
variables['property_type'] = variables['property_type'].apply(lambda x: x if x == 'Entire condo' else 'Other')
variables['property_type'].value_counts()

property_type
Other           16010
Entire condo    15198
Name: count, dtype: int64

In [50]:
variables['room_type'].value_counts()

room_type
Entire home/apt    27460
Private room        3319
Hotel room           407
Shared room           22
Name: count, dtype: int64

In [51]:
variables['room_type'] = variables['room_type'].apply(lambda x: x if x == 'Entire home/apt' else 'Other')
variables['room_type'].value_counts()

room_type
Entire home/apt    27460
Other               3748
Name: count, dtype: int64

In [52]:
variables['license'] = variables['license'].apply(lambda x: 'Sin licencia' if x == 'Sin licencia' else 'Con licencia')
variables['license'].value_counts()

license
Con licencia    25640
Sin licencia     5568
Name: count, dtype: int64

In [53]:
variables['host_is_superhost'] = variables['host_is_superhost'].replace('n/c', 'f')

# Regresión logistica

### host_is_superhost

In [54]:
variables['host_is_superhost'].unique()

array(['t', 'f'], dtype=object)

In [55]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = variables[['number_of_reviews_ltm']]
Var_Dep = variables['host_is_superhost']

# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

#Calculo la precisión del modelo
precision1 = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision1)
precision2 = precision_score(y_test, y_pred, average="binary", pos_label="t")
print(precision2)

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo

sensibilidad1 = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad1)
sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label="t")
print(sensibilidad2)

#Calculo el Puntaje F1 del modelo
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1)
  

Matriz de Confusión:
[[3990  909]
 [2916 1548]]
Precisión del modelo:
0.577758470894874
0.63003663003663
Exactitud del modelo:
0.5914770906760654
Sensibilidad del modelo:
0.8144519289650949
0.3467741935483871
Puntaje F1 del modelo:
0.44733420026007803


### host_has_profile_pic

In [56]:
variables['host_has_profile_pic'].unique()


array(['t', 'f'], dtype=object)

In [57]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep1 = variables[['number_of_reviews_ltm']]
Var_Dep1 = variables['host_has_profile_pic']

# Redefinimos las variables
X1 = Vars_Indep
y1 = Var_Dep

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X1_train = escalar.fit_transform(X1_train)
X1_test = escalar.transform(X1_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X1_train, y1_train)

#Realizamos una predicción
y_pred1 = algoritmo.predict(X1_test)

#Verifico la matriz de Confusión
matriz1 = confusion_matrix(y1_test, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

#Calculo la precisión del modelo
precision3 = precision_score(y1_test, y_pred1, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision3)
precision4 = precision_score(y1_test, y_pred1, average="binary", pos_label="t")
print(precision4)

exactitud1 = accuracy_score(y1_test, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

#Calculo la sensibilidad del modelo

sensibilidad3 = recall_score(y1_test, y_pred1, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad3)
sensibilidad4 = recall_score(y1_test, y_pred1, average="binary", pos_label="t")
print(sensibilidad4)

#Calculo el Puntaje F1 del modelo
puntajef1_1 = f1_score(y1_test, y_pred1, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1_1)

Matriz de Confusión:
[[3932  902]
 [2901 1628]]
Precisión del modelo:
0.5754427045221718
0.6434782608695652
Exactitud del modelo:
0.5938267649257717
Sensibilidad del modelo:
0.8134050475796442
0.3594612497240009
Puntaje F1 del modelo:
0.46125513528828443


### host_identity_verified

In [58]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep2 = variables[['number_of_reviews_ltm']]
Var_Dep2 = variables['host_identity_verified']

# Redefinimos las variables
X2 = Vars_Indep2
y2 = Var_Dep2

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X2_train = escalar.fit_transform(X2_train)
X2_test = escalar.transform(X2_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X2_train, y2_train)

#Realizamos una predicción
y_pred2 = algoritmo.predict(X2_test)

#Verifico la matriz de Confusión
matriz2 = confusion_matrix(y2_test, y_pred2)
print('Matriz de Confusión:')
print(matriz2)

#Calculo la precisión del modelo
precision5 = precision_score(y2_test, y_pred2, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision5)
precision6 = precision_score(y2_test, y_pred2, average="binary", pos_label="t")
print(precision6)

exactitud2 = accuracy_score(y2_test, y_pred2)
print('Exactitud del modelo:')
print(exactitud2)

#Calculo la sensibilidad del modelo

sensibilidad5 = recall_score(y2_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad5)
sensibilidad6 = recall_score(y2_test, y_pred2, average="binary", pos_label="t")
print(sensibilidad6)

#Calculo el Puntaje F1 del modelo
puntajef1_2 = f1_score(y2_test, y_pred2, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1_2)

Matriz de Confusión:
[[   0 1276]
 [   0 8087]]
Precisión del modelo:
0.0
0.8637188935170351
Exactitud del modelo:
0.8637188935170351
Sensibilidad del modelo:
0.0
1.0
Puntaje F1 del modelo:
0.9268767908309455


c:\Users\kile4\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [59]:
#Aplicación de Over Sampling con SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X2_train, y2_train)

#Entrenamos el modelo sobremuestreado
algoritmo_Over_sampling = LogisticRegression()
algoritmo_Over_sampling.fit(X_resampled, y_resampled)

#Realizamos una predicción
y_pred_over_sampling = algoritmo_Over_sampling.predict(X2_test) 

matriz_sobremuestreo = confusion_matrix(y2_test, y_pred_over_sampling)
print('Matriz de Confusión:')
print(matriz_sobremuestreo)

#Calculo la precisión del modelo
precision_over_sampling_f = precision_score(y2_test, y_pred_over_sampling, average="binary", pos_label="f")
precision_over_sampling_t = precision_score(y2_test, y_pred_over_sampling, average="binary", pos_label="t")
print('Precisión del modelo:')
print(precision_over_sampling_f)
print(precision_over_sampling_t)

exactitud_over_sampling = accuracy_score(y2_test, y_pred_over_sampling)
print('Exactitud del modelo:')  
print(exactitud_over_sampling)  

#Calculo la sensibilidad del modelo
sensibilidad_over_sampling_f = recall_score(y2_test, y_pred_over_sampling, average="binary", pos_label="f")
sensibilidad_over_sampling_t = recall_score(y2_test, y_pred_over_sampling, average="binary", pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad_over_sampling_f)
print(sensibilidad_over_sampling_t)

#Calculo el Puntaje F1 del modelo
puntajef1_over_sampling = f1_score(y2_test, y_pred_over_sampling, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1_over_sampling)

Matriz de Confusión:
[[ 394  882]
 [2271 5816]]
Precisión del modelo:
0.14784240150093808
0.8683189011645267
Exactitud del modelo:
0.6632489586670939
Sensibilidad del modelo:
0.30877742946708464
0.7191789291455423
Puntaje F1 del modelo:
0.7867433209333784


### instant_bookable

In [60]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep3 = variables[['number_of_reviews_ltm']]
Var_Dep3 = variables['instant_bookable']

# Redefinimos las variables
X3 = Vars_Indep3
y3 = Var_Dep3

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X3_train = escalar.fit_transform(X3_train)
X3_test = escalar.transform(X3_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X3_train, y3_train)

#Realizamos una predicción
y_pred3 = algoritmo.predict(X3_test)

#Verifico la matriz de Confusión
matriz3 = confusion_matrix(y3_test, y_pred3)
print('Matriz de Confusión:')
print(matriz3)

#Calculo la precisión del modelo
precision7 = precision_score(y3_test, y_pred3, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision7)
precision8 = precision_score(y3_test, y_pred3, average="binary", pos_label="t")
print(precision8)

exactitud3 = accuracy_score(y3_test, y_pred3)
print('Exactitud del modelo:')
print(exactitud3)

#Calculo la sensibilidad del modelo

sensibilidad7 = recall_score(y3_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad7)
sensibilidad8 = recall_score(y3_test, y_pred3, average="binary", pos_label="t")
print(sensibilidad8)

#Calculo el Puntaje F1 del modelo
puntajef1_3 = f1_score(y3_test, y_pred3, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1_3)

Matriz de Confusión:
[[   0 4361]
 [   0 5002]]
Precisión del modelo:
0.0
0.5342304816832212
Exactitud del modelo:
0.5342304816832212
Sensibilidad del modelo:
0.0
1.0
Puntaje F1 del modelo:
0.6964148973198747


c:\Users\kile4\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [61]:
#Método de reponderación de clases
algoritmo_reponderacion = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo
algoritmo_reponderacion.fit(X3_train, y3_train)

#Realizamos una predicción
y_pred3_reponderacion = algoritmo_reponderacion.predict(X3_test)

#Verifico la matriz de Confusión
matriz3_reponderacion = confusion_matrix(y3_test, y_pred3_reponderacion)
print('Matriz de Confusión:')
print(matriz3_reponderacion)

#Calculo la precisión del modelo
precision7_reponderacion = precision_score(y3_test, y_pred3_reponderacion, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision7_reponderacion)
precision8_reponderacion = precision_score(y3_test, y_pred3_reponderacion, average="binary", pos_label="t")
print(precision8_reponderacion)

exactitud3_reponderacion = accuracy_score(y3_test, y_pred3_reponderacion)
print('Exactitud del modelo:')
print(exactitud3_reponderacion)

#Calculo la sensibilidad del modelo

sensibilidad7_reponderacion = recall_score(y3_test, y_pred3_reponderacion, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad7_reponderacion)
sensibilidad8_reponderacion = recall_score(y3_test, y_pred3_reponderacion, average="binary", pos_label="t")
print(sensibilidad8_reponderacion)

#Calculo el Puntaje F1 del modelo
puntajef1_3_reponderacion = f1_score(y3_test, y_pred3_reponderacion, average="binary", pos_label="t")
print('Puntaje F1 del modelo:') 
print(puntajef1_3_reponderacion)



Matriz de Confusión:
[[3198 1163]
 [3546 1456]]
Precisión del modelo:
0.47419928825622776
0.5559373806796487
Exactitud del modelo:
0.4970629071878671
Sensibilidad del modelo:
0.7333180463196515
0.29108356657337064
Puntaje F1 del modelo:
0.3821020863403753


### host_response_time

In [62]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep4 = variables[['number_of_reviews_ltm']]
Var_Dep4 = variables['host_response_time']

# Redefinimos las variables
X4 = Vars_Indep
y4 = Var_Dep

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X4_train = escalar.fit_transform(X4_train)
X4_test = escalar.transform(X4_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X4_train, y4_train)

#Realizamos una predicción
y_pred4 = algoritmo.predict(X4_test)

#Verifico la matriz de Confusión
matriz4 = confusion_matrix(y4_test, y_pred4)
print('Matriz de Confusión:')
print(matriz4)

#Calculo la precisión del modelo
precision9 = precision_score(y4_test, y_pred4, average="binary", pos_label="f")
print('Precisión del modelo:')
print(precision9)
precision10 = precision_score(y4_test, y_pred4, average="binary", pos_label="t")
print(precision10)

exactitud4 = accuracy_score(y4_test, y_pred4)
print('Exactitud del modelo:')
print(exactitud4)

#Calculo la sensibilidad del modelo

sensibilidad9 = recall_score(y4_test, y_pred4, average="binary", pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad9)
sensibilidad10 = recall_score(y4_test, y_pred4, average="binary", pos_label="t")
print(sensibilidad10)

#Calculo el Puntaje F1 del modelo
puntajef1_4 = f1_score(y4_test, y_pred4, average="binary", pos_label="t")
print('Puntaje F1 del modelo:')
print(puntajef1_4)

Matriz de Confusión:
[[4022  903]
 [2840 1598]]
Precisión del modelo:
0.5861264937336054
0.6389444222311076
Exactitud del modelo:
0.6002349674249706
Sensibilidad del modelo:
0.8166497461928934
0.3600721045515998
Puntaje F1 del modelo:
0.46058509871739445


### host_response_rate	

In [63]:
variables['host_response_rate'].value_counts()

host_response_rate
90% o más       27542
Menos de 90%     3666
Name: count, dtype: int64

In [64]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep5 = variables[['calculated_host_listings_count']]
Var_Dep5 = variables['host_response_rate']

# Redefinimos las variables
X5 = Vars_Indep5
y5 = Var_Dep5

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X5_train = escalar.fit_transform(X5_train)
X5_test = escalar.transform(X5_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X5_train, y5_train)

#Realizamos una predicción
y_pred5 = algoritmo.predict(X5_test)

#Verifico la matriz de Confusión
matriz5 = confusion_matrix(y5_test, y_pred5)
print('Matriz de Confusión:')
print(matriz5)

#Calculo la precisión del modelo
precision11 = precision_score(y5_test, y_pred5, average="binary", pos_label="90% o más")
print('Precisión del modelo:')
print(precision11)
precision12 = precision_score(y5_test, y_pred5, average="binary", pos_label="Menos de 90%")
print(precision12)

exactitud5 = accuracy_score(y5_test, y_pred5)
print('Exactitud del modelo:')
print(exactitud5)

#Calculo la sensibilidad del modelo

sensibilidad11 = recall_score(y5_test, y_pred5, average="binary", pos_label="90% o más")
print('Sensibilidad del modelo:')
print(sensibilidad11)
sensibilidad12 = recall_score(y5_test, y_pred5, average="binary", pos_label="Menos de 90%")
print(sensibilidad12)

#Calculo el Puntaje F1 del modelo
puntajef1_5 = f1_score(y5_test, y_pred5, average="binary", pos_label="90% o más")
print('Puntaje F1 del modelo:')
print(puntajef1_5)

Matriz de Confusión:
[[8255    0]
 [1108    0]]
Precisión del modelo:
0.8816618605147922
0.0
Exactitud del modelo:
0.8816618605147922
Sensibilidad del modelo:
1.0
0.0
Puntaje F1 del modelo:
0.9371097740946759


c:\Users\kile4\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [65]:
#Método de reponderación de clases
algoritmo_reponderacion = LogisticRegression(class_weight='balanced')

#Entrenamos el modelo
algoritmo_reponderacion.fit(X5_train, y5_train)

#Realizamos una predicción
y_pred_reponderacion5 = algoritmo_reponderacion.predict(X5_test)

#Verifico la matriz de Confusión
matriz_reponderacion = confusion_matrix(y5_test, y_pred_reponderacion5)
print('Matriz de Confusión:')
print(matriz_reponderacion)

#Calculo la precisión del modelo
precision_reponderacion_1 = precision_score(y5_test, y_pred_reponderacion5, average="binary", pos_label="90% o más")
print('Precisión del modelo:')
print(precision_reponderacion_1)
precision_reponderacion_2 = precision_score(y5_test, y_pred_reponderacion5  , average="binary", pos_label="Menos de 90%")
print(precision_reponderacion_2)

exactitud_reponderacion2 = accuracy_score(y5_test, y_pred_reponderacion5)
print('Exactitud del modelo:')
print(exactitud_reponderacion2)

#Calculo la sensibilidad del modelo

sensibilidad_reponderacion_1 = recall_score(y5_test, y_pred_reponderacion5, average="binary", pos_label="90% o más")
print('Sensibilidad del modelo:')
print(sensibilidad_reponderacion_1)
sensibilidad_reponderacion_2 = recall_score(y5_test, y_pred_reponderacion5, average="binary", pos_label="Menos de 90%")
print(sensibilidad_reponderacion_2)

#Calculo el Puntaje F1 del modelo
puntaje_f1_reponderacion2 = f1_score(y5_test, y_pred_reponderacion5, average="binary", pos_label="90% o más")
print('Puntaje F1 del modelo:')
print(puntaje_f1_reponderacion2)

Matriz de Confusión:
[[3683 4572]
 [ 245  863]]
Precisión del modelo:
0.9376272912423625
0.15878564857405703
Exactitud del modelo:
0.485528142689309
Sensibilidad del modelo:
0.4461538461538462
0.7788808664259927
Puntaje F1 del modelo:
0.6046129853073956


### host_acceptance_rate	

In [66]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep6 = variables[['estimated_occupancy_l365d']]
Var_Dep6 = variables['host_acceptance_rate']

# Redefinimos las variables
X6 = Vars_Indep6
y6 = Var_Dep6

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X6_train = escalar.fit_transform(X6_train)
X6_test = escalar.transform(X6_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X6_train, y6_train)

#Realizamos una predicción
y_pred6 = algoritmo.predict(X6_test)

#Verifico la matriz de Confusión
matriz6 = confusion_matrix(y6_test, y_pred6)
print('Matriz de Confusión:')
print(matriz6)

#Calculo la precisión del modelo
precision13 = precision_score(y6_test, y_pred6, average="binary", pos_label="80% o más")
print('Precisión del modelo:')
print(precision13)
precision14 = precision_score(y6_test, y_pred6, average="binary", pos_label="Menos de 80%")
print(precision14)

exactitud6 = accuracy_score(y6_test, y_pred6)
print('Exactitud del modelo:')
print(exactitud6)

#Calculo la sensibilidad del modelo

sensibilidad13 = recall_score(y6_test, y_pred6, average="binary", pos_label="80% o más")
print('Sensibilidad del modelo:')
print(sensibilidad13)
sensibilidad14 = recall_score(y6_test, y_pred6, average="binary", pos_label="Menos de 80%")
print(sensibilidad14)

#Calculo el Puntaje F1 del modelo
puntajef1_6 = f1_score(y6_test, y_pred6, average="binary", pos_label="80% o más")
print('Puntaje F1 del modelo:')
print(puntajef1_6)

Matriz de Confusión:
[[7095    0]
 [2268    0]]
Precisión del modelo:
0.7577699455302788
0.0
Exactitud del modelo:
0.7577699455302788
Sensibilidad del modelo:
1.0
0.0
Puntaje F1 del modelo:
0.8621946773605541


c:\Users\kile4\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [67]:
#Método de reponderación de clases
algoritmo_reponderacion = LogisticRegression(class_weight='balanced')
#Entrenamos el modelo
algoritmo_reponderacion.fit(X6_train, y6_train)
#Realizamos una predicción
y_pred6 = algoritmo_reponderacion.predict(X6_test)
#Verifico la matriz de Confusión
matriz6 = confusion_matrix(y6_test, y_pred6)
print('Matriz de Confusión:')
print(matriz6)
#Calculo la precisión del modelo
precision13 = precision_score(y6_test, y_pred6, average="binary", pos_label="80% o más")
print('Precisión del modelo:')      
print(precision13)
precision14 = precision_score(y6_test, y_pred6, average="binary", pos_label="Menos de 80%")
print(precision14)      
exactitud6 = accuracy_score(y6_test, y_pred6)
print('Exactitud del modelo:')
print(exactitud6)
#Calculo la sensibilidad del modelo
sensibilidad13 = recall_score(y6_test, y_pred6, average="binary", pos_label="80% o más")
print('Sensibilidad del modelo:')
print(sensibilidad13)
sensibilidad14 = recall_score(y6_test, y_pred6, average="binary", pos_label="Menos de 80%")
print(sensibilidad14)

Matriz de Confusión:
[[3179 3916]
 [ 301 1967]]
Precisión del modelo:
0.9135057471264367
0.33435322114567395
Exactitud del modelo:
0.5496101676812988
Sensibilidad del modelo:
0.448062015503876
0.8672839506172839


### property_type	

In [74]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = variables[['price']]
Var_Dep = variables['property_type']

# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

#Calculo la precisión del modelo
precision1 = precision_score(y_test, y_pred, average="binary", pos_label="Entire condo")
print('Precisión del modelo:')
print(precision1)
precision2 = precision_score(y_test, y_pred, average="binary", pos_label="Other")
print(precision2)

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo

sensibilidad1 = recall_score(y_test, y_pred, average="binary", pos_label="Entire condo")
print('Sensibilidad del modelo:')
print(sensibilidad1)
sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label="Other")
print(sensibilidad2)

#Calculo el Puntaje F1 del modelo
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="Entire condo")
print('Puntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión:
[[ 542 3979]
 [1027 3815]]
Precisión del modelo:
0.3454429572976418
0.489479086476777
Exactitud del modelo:
0.4653423048168322
Sensibilidad del modelo:
0.11988498119884981
0.7878975629904998
Puntaje F1 del modelo:
0.1779967159277504


### room_type

In [75]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep9 = variables[['price']]
Var_Dep9 = variables['room_type']

# Redefinimos las variables
X9 = Vars_Indep9
y9 = Var_Dep9

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y9, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X9_train = escalar.fit_transform(X9_train)
X9_test = escalar.transform(X9_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X9_train, y9_train)

#Realizamos una predicción
y_pred9 = algoritmo.predict(X9_test)

#Verifico la matriz de Confusión
matriz9 = confusion_matrix(y9_test, y_pred9)
print('Matriz de Confusión:')
print(matriz9)

#Calculo la precisión del modelo
precision20 = precision_score(y9_test, y_pred9, average="binary", pos_label="Entire home/apt")
print('Precisión del modelo:')
print(precision20)
precision21 = precision_score(y9_test, y_pred9, average="binary", pos_label="Other")
print(precision21)

exactitud9 = accuracy_score(y9_test, y_pred9)
print('Exactitud del modelo:')
print(exactitud9)

#Calculo la sensibilidad del modelo

sensibilidad20 = recall_score(y9_test, y_pred9, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad20)
sensibilidad21 = recall_score(y9_test, y_pred9, average="binary", pos_label="Other")
print(sensibilidad21)

#Calculo el Puntaje F1 del modelo
puntajef1_9 = f1_score(y9_test, y_pred9, average="binary", pos_label="Other")
print('Puntaje F1 del modelo:')
print(puntajef1_9)

Matriz de Confusión:
[[8226   33]
 [1102    2]]
Precisión del modelo:
0.8818610634648371
0.05714285714285714
Exactitud del modelo:
0.8787781693901527
Sensibilidad del modelo:
0.9960043588812205
0.0018115942028985507
Puntaje F1 del modelo:
0.003511852502194908


In [76]:
#Método de sobremuestreo con SMOTE
smote = SMOTE()
X9_resampled, y9_resampled = smote.fit_resample(X9_train, y9_train)         
#Entrenamos el modelo sobremuestreado
algoritmo_Over_sampling = LogisticRegression()
algoritmo_Over_sampling.fit(X9_resampled, y9_resampled)
#Realizamos una predicción
y_pred_over_sampling9 = algoritmo_Over_sampling.predict(X9_test)      
#Verifico la matriz de Confusión
matriz_sobremuestreo9 = confusion_matrix(y9_test, y_pred_over_sampling9)
print('Matriz de Confusión:')
print(matriz_sobremuestreo9)
#Calculo la precisión del modelo
precision_over_sampling_20 = precision_score(y9_test, y_pred_over_sampling9, average="binary", pos_label="Entire home/apt")
print('Precisión del modelo:')
print(precision_over_sampling_20)
precision_over_sampling_21 = precision_score(y9_test, y_pred_over_sampling9, average="binary", pos_label="Other")
print(precision_over_sampling_21)
exactitud_over_sampling9 = accuracy_score(y9_test, y_pred_over_sampling9)
print('Exactitud del modelo:')  
print(exactitud_over_sampling9)  
#Calculo la sensibilidad del modelo
sensibilidad_over_sampling_20 = recall_score(y9_test, y_pred_over_sampling9, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad_over_sampling_20)
sensibilidad_over_sampling_21 = recall_score(y9_test, y_pred_over_sampling9, average="binary", pos_label="Other")
print(sensibilidad_over_sampling_21)

Matriz de Confusión:
[[6372 1887]
 [ 595  509]]
Precisión del modelo:
0.9145973876847998
0.2124373956594324
Exactitud del modelo:
0.7349140232831357
Sensibilidad del modelo:
0.7715219760261532
0.46105072463768115


### 	license

In [79]:
# Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = variables[['number_of_reviews_ltm']]
Var_Dep = variables['license']

# Redefinimos las variables
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 

#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

#Calculo la precisión del modelo
precision1 = precision_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Precisión del modelo:')
print(precision1)
precision2 = precision_score(y_test, y_pred, average="binary", pos_label="Sin licencia")
print(precision2)

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo

sensibilidad1 = recall_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Sensibilidad del modelo:')
print(sensibilidad1)
sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label="Sin licencia")
print(sensibilidad2)

#Calculo el Puntaje F1 del modelo
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Puntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión:
[[7699    0]
 [1664    0]]
Precisión del modelo:
0.8222791840222151
0.0
Exactitud del modelo:
0.8222791840222151
Sensibilidad del modelo:
1.0
0.0
Puntaje F1 del modelo:
0.9024733325518697


c:\Users\kile4\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [80]:
#Método de reponderación de clases
algoritmo_reponderacion = LogisticRegression(class_weight='balanced')
#Entrenamos el modelo
algoritmo_reponderacion.fit(X_train, y_train)
#Realizamos una predicción
y_pred = algoritmo_reponderacion.predict(X_test)
#Verifico la matriz de Confusión
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)
#Calculo la precisión del modelo
precision1 = precision_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Precisión del modelo:')
print(precision1)
precision2 = precision_score(y_test, y_pred, average="binary", pos_label="Sin licencia")
print(precision2)
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
#Calculo la sensibilidad del modelo
sensibilidad1 = recall_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Sensibilidad del modelo:')
print(sensibilidad1)
sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label="Sin licencia")
print(sensibilidad2)
#Calculo el Puntaje F1 del modelo
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="Con licencia")
print('Puntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión:
[[2554 5145]
 [ 354 1310]]
Precisión del modelo:
0.8782668500687758
0.2029434546862897
Exactitud del modelo:
0.41268824094841394
Sensibilidad del modelo:
0.3317313936874919
0.7872596153846154
Puntaje F1 del modelo:
0.48156877533704157
